In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import fsspec

#import earthnet_minicuber
from pyproj import Transformer
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

from pathlib import Path

import pystac_client
import rasterio
import time

from copy import deepcopy

from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

In [2]:
SHORT_TO_LONG_NAMES = {
    't2m': '2m_temperature', 
    'slhf': 'surface_latent_heat_flux',
    'ssr': 'surface_net_solar_radiation', 
    'msl': 'mean_sea_level_pressure', 
    'sshf': 'surface_sensible_heat_flux',
    'e': 'evaporation', 
    'tp': 'total_precipitation',
}

In [3]:
bands = ['t2m', 'slhf', 'ssr', 'msl', 'sshf', 'e', 'tp']
daily_aggregation_types = ["mean", "min", "max"]
leadtime_months = [0,1,2,3,4,5]
ensemble_realization_numbers = [0,1,2,3,20,40,45,50]
ensemble_aggregation = ["mean","median","min", "max"]
zarrpath = None
zarrurl = 'https://storage.de.cloud.ovh.net/v1/AUTH_84d6da8e37fe4bb5aea18902da8c1170/uc1-africa/ecmwf_africa_1d0_6hourly.zarr'

lon_lat = (33.598946, -5.087414)
xy_shape = (256, 256)
resolution = 20
time_interval = "2021-01-01/2021-03-31"

In [4]:
utm_epsg = int(query_utm_crs_info(
            datum_name="WGS 84",
            area_of_interest=AreaOfInterest(lon_lat[0], lon_lat[1], lon_lat[0], lon_lat[1])
        )[0].code)

transformer = Transformer.from_crs(4326, utm_epsg, always_xy=True)
x_center, y_center = transformer.transform(*lon_lat)
nx, ny = xy_shape
x_left, x_right = x_center - resolution * (nx//2), x_center + resolution * (nx//2)
y_top, y_bottom = y_center + resolution * (ny//2), y_center - resolution * (ny//2)

bbox = transformer.transform_bounds(x_left, y_bottom, x_right, y_top, direction = 'INVERSE')

In [11]:
if zarrurl:
    ecmwf = xr.open_zarr(fsspec.get_mapper(zarrurl), consolidated=True)
elif zarrpath:
    ecmwf = xr.open_zarr(zarrpath, consolidated = False)

ecmwf = ecmwf[bands]

In [12]:
ecmwf

<xarray.Dataset>
Dimensions:     (number: 51, time: 60, step: 719, latitude: 81, longitude: 81)
Coordinates:
  * latitude    (latitude) float64 40.0 39.0 38.0 37.0 ... -38.0 -39.0 -40.0
  * longitude   (longitude) float64 -25.0 -24.0 -23.0 -22.0 ... 53.0 54.0 55.0
  * number      (number) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49 50
  * step        (step) timedelta64[ns] 0 days 06:00:00 ... 179 days 18:00:00
    surface     float64 ...
  * time        (time) datetime64[ns] 2017-01-01 2017-02-01 ... 2021-12-01
    valid_time  (time, step) datetime64[ns] dask.array<chunksize=(30, 719), meta=np.ndarray>
Data variables:
    t2m         (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    slhf        (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    ssr         (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    msl         (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    sshf        (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    e           (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    tp          (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>

In [7]:
center_lon = (bbox[0] + bbox[2])/2
center_lat = (bbox[1] + bbox[3])/2
ecmwf = ecmwf.sel(latitude = center_lat, longitude = center_lon, method = "nearest").drop_vars(["latitude", "longitude"])

/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(
/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/xarray/core/indexes.py:234: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  indexer = self.index.get_loc(


In [19]:
ecmwf

<xarray.Dataset>
Dimensions:     (number: 51, time: 60, step: 719, latitude: 81, longitude: 81)
Coordinates:
  * latitude    (latitude) float64 40.0 39.0 38.0 37.0 ... -38.0 -39.0 -40.0
  * longitude   (longitude) float64 -25.0 -24.0 -23.0 -22.0 ... 53.0 54.0 55.0
  * number      (number) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49 50
  * step        (step) timedelta64[ns] 0 days 06:00:00 ... 179 days 18:00:00
    surface     float64 ...
  * time        (time) datetime64[ns] 2017-01-01 2017-02-01 ... 2021-12-01
    valid_time  (time, step) datetime64[ns] dask.array<chunksize=(30, 719), meta=np.ndarray>
Data variables:
    t2m         (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    slhf        (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    ssr         (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    msl         (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    sshf        (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    e           (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>
    tp          (number, time, step, latitude, longitude) float32 dask.array<chunksize=(1, 1, 719, 20, 20), meta=np.ndarray>

In [9]:
agg_ecmwf_collector = []
for daily_aggregation_type in daily_aggregation_types:
    if daily_aggregation_type == "mean":
        curr_agg_ecmwf = ecmwf.groupby("valid_time.date").mean("stacked_time_step").rename({"date": "time"})
    elif daily_aggregation_type == "min":
        curr_agg_ecmwf = ecmwf.groupby("valid_time.date").min("stacked_time_step").rename({"date": "time"})
    elif daily_aggregation_type == "max":
        curr_agg_ecmwf = ecmwf.groupby("valid_time.date").max("stacked_time_step").rename({"date": "time"})
    elif daily_aggregation_type == "median":
        curr_agg_ecmwf = ecmwf.groupby("valid_time.date").median("stacked_time_step").rename({"date": "time"})
    elif daily_aggregation_type == "std":
        curr_agg_ecmwf = ecmwf.groupby("valid_time.date").std("stacked_time_step").rename({"date": "time"})
    else:
        continue

    curr_agg_ecmwf["step"] = np.array([str(d) for d in curr_agg_ecmwf.time.values], dtype="datetime64[D]")
    curr_agg_ecmwf = curr_agg_ecmwf.rename({b: f"ecmwf_{b}_{daily_aggregation_type}" for b in bands})
    agg_ecmwf_collector.append(curr_agg_ecmwf)

agg_ecmwf = xr.merge(agg_ecmwf_collector)

agg_ecmwf = agg_ecmwf.sel(step = slice(time_interval[:10], time_interval[-10:]))

for b in bands:
    for a in daily_aggregation_types:

        agg_ecmwf[f"ecmwf_{b}_{a}"].attrs = {
            "provider": "ECMFWF seasonal weather forecasts",
            "interpolation_type": "linear",
            "description": f"{SHORT_TO_LONG_NAMES[b]} 6-hourly data aggregated by {a}"
        }

/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/crequ/anaconda3/envs/HS_UC1/lib/python3.9/site-packages/numpy/core/_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, kee

In [22]:
agg_ecmwf.sel(valid_time = slice(time_interval[:10], time_interval[-10:]))

KeyError: 'valid_time is not a valid dimension or coordinate'

In [10]:
agg_ecmwf

<xarray.Dataset>
Dimensions:          (time: 1975, number: 51, step: 90)
Coordinates:
  * number           (number) int64 0 1 2 3 4 5 6 7 ... 43 44 45 46 47 48 49 50
    surface          float64 0.0
  * time             (time) object 2017-01-01 2017-01-02 ... 2022-05-29
  * step             (step) datetime64[ns] 2021-01-01 2021-01-02 ... 2021-03-31
Data variables: (12/21)
    ecmwf_t2m_mean   (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_slhf_mean  (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_ssr_mean   (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_msl_mean   (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_sshf_mean  (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_e_mean     (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ...               ...
    ecmwf_slhf_max   (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_ssr_max    (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_msl_max    (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_sshf_max   (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_e_max      (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ecmwf_tp_max     (time, number) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>

In [ ]:
ecmwf

<xarray.Dataset>
Dimensions:     (number: 51, time: 60, step: 719)
Coordinates:
  * number      (number) int64 0 1 2 3 4 5 6 7 8 ... 42 43 44 45 46 47 48 49 50
  * step        (step) timedelta64[ns] 0 days 06:00:00 ... 179 days 18:00:00
    surface     float64 ...
  * time        (time) datetime64[ns] 2017-01-01 2017-02-01 ... 2021-12-01
    valid_time  (time, step) datetime64[ns] dask.array<chunksize=(30, 719), meta=np.ndarray>
Data variables:
    t2m         (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>
    slhf        (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>
    ssr         (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>
    msl         (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>
    sshf        (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>
    e           (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>
    tp          (number, time, step) float32 dask.array<chunksize=(1, 1, 719), meta=np.ndarray>